In [1]:
%matplotlib  qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
import numpy as np
import pyxem as pxm
import diffpy

from hyperspy.signals import Signal2D

from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from diffsims.generators.zap_map_generator import get_rotation_from_z_to_direction
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.rotation_list_generators import get_beam_directions_grid

from pyxem.generators.indexation_generator import IndexationGenerator

from orix.quaternion import symmetry, Orientation, Rotation
from orix.vector import Vector3d, Miller
from orix.crystal_map import Phase
#from pyxem.utils.indexation_utils import peaks_from_best_template


In [31]:
dp = hs.load(r'C:\Users\ursulal\OneDrive - NTNU\2023_SIDI_project_ELKEM\20240104_SIDILam1\20240104_114935\cropped_centered.hspy')

In [32]:
roi = hs.roi.RectangularROI(left=30, right=100, top=20, bottom=100)
dp.plot(norm='symlog')
roi.add_widget(dp)


In [33]:
dp = roi(dp)

In [23]:
rebinned = dp.rebin(scale=(1, 1, 2,2))

In [18]:
dp.plot()

In [19]:
rebinned.plot()

In [ ]:
dp = rebinned # to test things - binning in signal space

In [34]:
dp.axes_manager[0].scale = 100.47
dp.axes_manager[1].scale = 100.47
dp.axes_manager[2].scale = 1/100.47
dp.axes_manager[3].scale = 1/100.47
dp.axes_manager[0].units = "A"
dp.axes_manager[1].units = "A"
dp.axes_manager[2].units = "1/A"
dp.axes_manager[3].units = "1/A"
dp.axes_manager

Navigation axis name,size,index,offset,scale,units
,9,0,3177.57,100.47,A
,9,0,1487.58,100.47,A
Signal axis name,size,,offset,scale,units
,256,,0.0,0.009953219866626855,1/A
,256,,0.0,0.009953219866626855,1/A


In [ ]:
%matplotlib  inline

## 1. Visualising the SPED dataset
A quick way to get an overview of all the diffraction spots within the dataset, is to plot the maximum intensity with repsect to detector position. We can also look at the average to get a quick impression of the dataset.

In [ ]:
dp_max = dp.max()
dp_mean = dp.mean()

The colormap used in matplotlib can be specified when plotting a hyperpsy signal. For demonstration purposes various of the perceptially uniform colormaps is used within this notebook. An overview of available colourmaps can be found on the Matplotlib pages: https://matplotlib.org/3.5.0/tutorials/colors/colormaps.html

In [ ]:
dp_max.plot(cmap='inferno', norm='log')
dp_mean.plot(cmap='inferno', norm='log')

Plot the SPED signal and navigate thorugh it to see the various precipitate reflections appearing.

In [ ]:
dp.plot(cmap='magma')

Plotting the patterns on a log scale makes it easier to see the weak precipitate Bragg spots. This can also be achieved by right clicking on the pattern and pressing 'l' on the keyboard.

In [ ]:
dp.plot(cmap='magma_r', norm='symlog')

The axes manager is useful to see the size of the dataset and the calibrations in both real and reciprocal space. Double check the scaling.

In [ ]:
dp.axes_manager

We save the calibration in reciprocal space to a variable, for easy use later.

In [38]:
calibration = dp.axes_manager[2].scale
calibration

0.009953219866626855

Virtual imaging can readily be done by defining a region of interest, which will be used as our integration window. We will use a circular aperture to obtan virtual STEM imaging. We can define a virtual bright field (VBF), virtual annular dark field (VADF), or a virtual dark field (VDF) aperture. For this dataset, the precipitates appear bright especially if a VADF aperture is placed in between the spots.



In [ ]:
roi = hs.roi.CircleROI(cx=0, cy=0, r=20*calibration, r_inner=10*calibration)

We can plot the integrated intensity from within the integration window, which will appear in a new window. You can use the navigator to move to a new scan position, and you can use the mouse to move the virtual aperture and change its size.



In [ ]:
dp.plot_integrated_intensity(roi)

The value of 'roi' is updated as you drag it around and change its size using the mouse. Print out the current 'roi'.

In [ ]:
roi

## Blob detection using scikit image

# Masking out background
Now that blob detection works, set the background outside the spts to -1

In [ ]:
len(dp.data)

In [35]:
import numpy as np
from skimage.feature import blob_log
from skimage import draw
from skimage.transform import rescale
from scipy.ndimage import gaussian_filter
import hyperspy.api as hs

# Parameters for blob detection
max_sigma = 30
threshold = 3
diffraction_patterns = dp.data
diffraction_patterns = gaussian_filter(dp.data, sigma = 1)

# Create an empty array to store the summed intensities
summed_intensities = np.empty((diffraction_patterns.shape[0], diffraction_patterns.shape[1]))

# Initialize processed_patterns with zero arrays instead of None
processed_patterns = np.zeros_like(diffraction_patterns)
for i in range(processed_patterns.shape[0]):
    for j in range(processed_patterns.shape[1]):
        processed_patterns[i, j] = np.zeros((256, 256))
        
# Process each diffraction pattern
for row in range(diffraction_patterns.shape[0]):
    for col in range(diffraction_patterns.shape[1]):
        pattern = diffraction_patterns[row, col].copy()  # Make a copy of the pattern

        # Detect blobs using LoG method
        blobs = blob_log(pattern, max_sigma=max_sigma, threshold=threshold)
        blobs[:, 2] = blobs[:, 2] * np.sqrt(2)  # Compute radii

        # Create a high resolution mask with the same shape as the pattern, initialized with -1
        mask = np.full_like(pattern, -1, dtype=np.float64)
        mask_high_res = rescale(mask, 10, order=0, mode='constant', cval=-1, clip=True, preserve_range=True)

        # Draw blobs onto the high resolution mask
        for blob in blobs:
            y, x, r = blob
            rr, cc = draw.disk((y*10, x*10), r*10, shape=mask_high_res.shape)
            mask_high_res[rr, cc] = 1  # Set the pixel values at the blob locations to 1 in the mask

        # Downsample the high resolution mask to the original resolution
        mask = rescale(mask_high_res, 0.1, order=1, mode='reflect', cval=0, clip=True, preserve_range=True)
        mask = np.round(mask).astype(np.int64)

        # Set the area outside the blobs to -1
        pattern[mask == -1] = -1

        # Only add the pattern to the processed_patterns array if it is not None
        if pattern is not None:
            processed_patterns[row, col] = pattern
            
        # Create an empty array to store the summed intensities
        summed_intensities = np.empty((len(dp.data), len(dp.data)))

        # Sum the intensities of the pattern and store it in the summed_intensities array
        for i in range(len(dp.data)):
            for j in range(len(dp.data)):
                summed_intensities[i, j] = np.sum(processed_patterns[i, j])

# Create a Signal2D object with the summed intensities as the navigation signal
navigator = hs.signals.Signal2D(summed_intensities)

# Create a Diffraction2D hyperspy object with the processed patterns as the signal
dp_blob_diffraction = hs.signals.Signal2D(processed_patterns)

# Transpose the axes of the Diffraction2D object
dp_blob_diffraction = dp_blob_diffraction.transpose(signal_axes=[2, 3], navigation_axes=[0, 1])

# Set the navigator of the Diffraction2D object
dp_blob_diffraction.plot(navigator=navigator)


In [ ]:
dp_blob_diffraction.save(r'C:\Users\ursulal\OneDrive - NTNU\2023_SIDI_project_ELKEM\20240104_SIDILam1\20240104_114935\dp_masked.hspy')

In [ ]:
dp = dp_blob_diffraction

In [12]:
dp_m = hs.load(r'C:\Users\ursulal\OneDrive - NTNU\2023_SIDI_project_ELKEM\20240104_SIDILam1\20240104_114935\dp_masked.hspy')#dp.plot()

In [13]:
dp_m.plot()

## An attempt of vectorization and parallelization - 240315

In [ ]:
import numpy as np
from skimage.feature import blob_log
from skimage import draw
from skimage.transform import rescale
import hyperspy.api as hs
from multiprocessing import Pool
from scipy.ndimage import gaussian_filter

# Parameters for blob detection
max_sigma = 30
threshold = 4
diffraction_patterns = dp.data
diffraction_patterns = gaussian_filter(dp.data, sigma = 1)

# Initialize processed_patterns with zero arrays instead of None
processed_patterns = np.zeros_like(diffraction_patterns)


In [ ]:
def process_pattern(pattern):
    # Detect blobs using LoG method
    blobs = blob_log(pattern, max_sigma=max_sigma, threshold=threshold)
    blobs[:, 2] = blobs[:, 2] * np.sqrt(2)  # Compute radii

    # Create a high resolution mask with the same shape as the pattern, initialized with -1
    mask = np.full_like(pattern, -1, dtype=np.float64)
    mask_high_res = rescale(mask, 10, order=0, mode='constant', cval=-1, clip=True, preserve_range=True)

    # Draw blobs onto the high resolution mask
    for blob in blobs:
        y, x, r = blob
        rr, cc = draw.disk((y*10, x*10), r*10, shape=mask_high_res.shape)
        mask_high_res[rr, cc] = 1  # Set the pixel values at the blob locations to 1 in the mask

    # Downsample the high resolution mask to the original resolution
    mask = rescale(mask_high_res, 0.1, order=1, mode='reflect', cval=0, clip=True, preserve_range=True)
    mask = np.round(mask).astype(np.int64)

    # Set the area outside the blobs to -1
    pattern[mask == -1] = -1

    return pattern


In [ ]:
# Create a pool of workers
with Pool() as p:
    # Flatten the diffraction_patterns array and apply the process_pattern function to each pattern
    processed_patterns = np.array(p.map(process_pattern, diffraction_patterns.reshape(-1, diffraction_patterns.shape[-1], diffraction_patterns.shape[-1])))


In [ ]:
# Reshape the processed_patterns array back to its original shape
processed_patterns = processed_patterns.reshape(diffraction_patterns.shape)

# Sum the intensities of the pattern and store it in the summed_intensities array
summed_intensities = np.sum(processed_patterns, axis=(2,3))

# Create a Signal2D object with the summed intensities as the navigation signal
navigator = hs.signals.Signal2D(summed_intensities)

# Create a Diffraction2D hyperspy object with the processed patterns as the signal
dp_blob_diffraction = hs.signals.Signal2D(processed_patterns)

# Transpose the axes of the Diffraction2D object
dp_blob_diffraction = dp_blob_diffraction.transpose(signal_axes=[2, 3], navigation_axes=[0, 1])

# Set the navigator of the Diffraction2D object
dp_blob_diffraction.plot(navigator=navigator)


In [ ]:
dp.find_peaks()

# Blob  detection using hyperspy map function 
It seems that the map function is slower

In [ ]:
import numpy as np
from skimage.feature import blob_log
from skimage import draw
from scipy.ndimage import gaussian_filter
import hyperspy.api as hs

# Parameters for blob detection
max_sigma = 30
threshold = 5
diffraction_patterns = gaussian_filter(dp.data, sigma = 1)# dp.data #

# the simulated patterns we need a mean radius 
r_list = []

# Define a function to process a diffraction pattern
def process_diffraction_pattern(pattern):
    # Detect blobs using LoG method
    blobs = blob_log(pattern, max_sigma=max_sigma, threshold=threshold)
    blobs[:, 2] = blobs[:, 2] * np.sqrt(2)  # Compute radii in the correct scale

    # Create a mask with the same shape as the pattern, initialized with -1
    mask = np.full_like(pattern, -1)

    # Draw blobs onto the mask
    for blob in blobs:
        y, x, r = blob
        r_list.append(r)
        rr, cc = draw.disk((y, x), r, shape=pattern.shape)
        mask[rr, cc] = 1  # Set the pixel values at the blob locations to 1 in the mask

    # Set the area outside the blobs to -1
    pattern[mask == -1] = -1

    return pattern

# Apply the function to each diffraction pattern in the dataset
dp.map(process_diffraction_pattern, inplace=True)

# Now let's plot the processed patterns with overlays
dp.plot()


## The mean value of the diffraction disks & mean value in 


In [ ]:
avg_r = np.mean(r_list)

## Creating template libraries
Sampling orientations and simulating diffraction
# Finding the in-plane rotation of the average pattern
Import the crystal information file (cif) for each of the  phases using diffpy. This file contains information on the space group, lattice parameters, atom sites, atom types and site occupations.

In [36]:
import diffpy
structure_aFe = diffpy.structure.loadStructure(r'C:\Users\ursulal\OneDrive - NTNU\2023_SIDI_project_ELKEM\cif\Fe_mp-13_conventional_standard.cif')
structure_FeC = diffpy.structure.loadStructure(r'C:\Users\ursulal\OneDrive - NTNU\2023_SIDI_project_ELKEM\cif\Fe3C_mp-510623_conventional_standard.cif')
structure_MnS = diffpy.structure.loadStructure(r'C:\Users\ursulal\OneDrive - NTNU\2023_SIDI_project_ELKEM\cif\MnS.cif')

Set up a diffraction generator using diffsims. This generator contains information on the microscope settings, most importantly the incident beam energy, and it will be used ater for simulating diffraction.

We also set up a generator that will be used in creating a library of simulated diffraction patterns. This library generator takes the diffraction generator as input.



First, we will focus on the average pattern from the whole dataset. This pattern shows the Fe matrix oriented to (or very close to) the zone axis <100>. We aim to simulate a diffraction pattern that matches this average pattern.

Create a structure library containing the phase identifier, the crystal structure and the orientations that will be used to simulate diffraction. For now, we will only lookt the crystalum oriented to Euler angles (0,0,0), which will leave the crystl oriented to the <100> zone axi In the context of crystallography and electron diffraction, the term “zone axis” refers to a direction in a crystal lattice. When we say a crystal is oriented along a certain zone axis, we mean that the electron beam is parallel to that direction.

So, if the Euler angles are (0,0,0), it means the crystal orientation is the same as the reference orientation. The specific zone axis corresponding to this orientation depends on the reference state of the crystal. For a cubic crystal system, if the reference state is aligned along the [100] direction, then the Euler angles (0,0,0) would correspond to the [100] zone a

This is suitable for materials, like Al, where [100] is the most used orientation. In case of Fe allotrpes it might be better to start with a random zone axis.
On 240112 the zone axis for testing the notebook is [111] and has the Euler angles (90., 54.736, 45.).s.s.

In [ ]:
from diffsims.libraries.structure_library import StructureLibrary
euler_100 = (90., 54.736, 45.) # <111> zone axis # (0., 0., 0.) # <100> zone axis 
library_Fe = StructureLibrary(identifiers = ['Fe'], 
                              structures = [structure_aFe], 
                              orientations = [[euler_100]])

# Cementite FeC

Define variables needed to simulate diffraction. We know the length of the detector and can use that to calculate how far into reciprocal space we need to simulate diffraction for ('reciprocal_radius'), in order to cover the whole detector, using Pythagoras.

We also need to determine the maximum excitation error in the diffraction simulation. This parameter can very roughly be interpreted as the relrod length ans is related to the inverse thickness of the specimen.



Calculate the library of simulated diffraction patterns, for the structure library defined above, which contained only the aFe and the orientation (0,0,0).



In [ ]:
diff_lib_Fe = lib_gen.get_diffraction_library(library_Fe,
                                              calibration=calibration,
                                              reciprocal_radius=reciprocal_radius,
                                              half_shape=half_shape,
                                              with_direct_beam=False,
                                              max_excitation_error=max_excitation_error)

This simulated library contains for each simulated pattern, the phase, the orientation, the center coordinates of the simulated Bragg spots, their intensities and their indices hkl.

In [ ]:
diff_lib_Fe

We pick out the first (and only) simulation and plot it.



In [ ]:
simulation_Fe = diff_lib_Fe['Fe']['simulations'][0]
#simulation_Fe.plot()


In [ ]:
simulation_Fe.indices

We see that the simulation and indices fit what was expected. However, we do not know the in-plane rotation of the spots, which we can calculate. This is done by calculating the polar transforms of both the pattern and the simulation and calculating their cross correlation coefficient as one is shifted over the other.

In [ ]:
from pyxem.utils import indexation_utils as iutls
a, c = iutls.get_in_plane_rotation_correlation(dp_mean.data, simulation_Fe)

Define the in-plane angle as the angle with the highest correlation score.

In [ ]:
in_plane_angle = a[np.argmax(c)]
in_plane_angle

We plot the simulated spots on top of the average pattern. With the correct in-plane rotation applied, there is

In [ ]:
from pyxem.utils import plotting_utils as putls
from matplotlib.colors import LogNorm
putls.plot_template_over_pattern(dp_mean.data,
                                 simulation_Fe,
                                 in_plane_angle=in_plane_angle,
                                 norm=LogNorm())

## Sampling orientations within the fundamental sector
Next we want to create a list of orientations sampled from the whole fundamental sector of the respective crystal system. This does not take into account the in-plane rotations, only the beam directions.

We set an angular resolution for creation of the grid of orientations.

In [ ]:
resolution = 1 # What does angular resolution mean?

In [ ]:
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
grid_aFe = get_beam_directions_grid(
    resolution=resolution, crystal_system = "cubic") # Ferrite
grid_FeC = get_beam_directions_grid(
    resolution=resolution, crystal_system = "orthorhombic") # FeC
grid_MnS = get_beam_directions_grid(
    resolution=resolution, crystal_system = "cubic") # MnS



Due to the various symmetries of the three lattice types, the length of the grids are quite different.



In [ ]:
print(len(grid_aFe), len(grid_FeC), len(grid_MnS))


Define the Laue group symmetry of each of the three phases. Here, Schoenflies notation is used. Take a look at the orix documentation to find an overview of the 32 point groups and the corresponding 11 Laue groups. https://orix.readthedocs.io/en/stable/inverse_pole_figures.html - the link is not working

In [ ]:
from orix.quaternion import symmetry, Orientation, Rotation
symmetry_aFe = symmetry.Oh
symmetry_FeC = symmetry.D2h
symmetry_MnS = symmetry.Oh

These contain all the allowed symmetry operations.

In [ ]:
symmetry_aFe
symmetry_FeC
symmetry_MnS

Calculate the orientations from the grid of Euler angles, using the respective symmetries as input. The orientations are given as quaternions. Note that "In orix, orientations and misorientations are distinguished from rotations only by the inclusion of a notion of symmetry." - this needs more documentation

In [ ]:
orientations_aFe = Orientation.from_euler(np.deg2rad(grid_aFe), symmetry=symmetry_aFe)
orientations_FeC = Orientation.from_euler(np.deg2rad(grid_FeC), symmetry=symmetry_FeC)
orientations_MnS = Orientation.from_euler(np.deg2rad(grid_MnS), symmetry=symmetry_MnS)

In [ ]:
orientations_aFe

Plot these orientations on the inverse pole figure (IPF).

In [ ]:
orientations_aFe.scatter('ipf')
orientations_FeC.scatter('ipf')
orientations_MnS.scatter('ipf')

Rotate the beam direction (optic axis) according to the crystal orientations.



In [ ]:

orientations_aFe_z = orientations_aFe * Vector3d.zvector() 
orientations_FeC_z = orientations_FeC * Vector3d.zvector()
orientations_MnS_z = orientations_MnS * Vector3d.zvector()

Plot the beam directions (the sampled zone axes) inside the upper hemisphere of the full [001] stereograpic projections 

In [ ]:
orientations_aFe_z.scatter(hemisphere="both") # 240205 error:'numpy.ndarray' object has no attribute 'scatter'
orientations_FeC_z.scatter(hemisphere="both")
orientations_MnS_z.scatter(hemisphere="both")

## 2c. Sampling orientations around a zone axis
Define each phase using the Phase class in orix, with the structure and symmetry as inpu

In [ ]:
from orix.crystal_map import Phase


In [42]:
phase_aFe = Phase(structure=structure_aFe, point_group=symmetry_aFe)
phase_FeC = Phase(structure=structure_FeC, point_group=symmetry_FeC)
phase_MnS = Phase(structure=structure_MnS, point_group=symmetry_MnS)

Since the precipitates have specific orientation relationships with the aluminium matrix, we know what zone axis they should be close to when the matrix is oriented to zone axis [001]. If there are several zone axis that are equivalent by symmetry, we choose those that lie within the fundamental zone, since the grid of Euler angles we defined earlier was for the fundamental zone.

In [ ]:
# Define the diffraction generator with a desired maximum excitation error
diff_gen = DiffractionGenerator(200, max_excitation_error=1/10)


In [ ]:
# Define the rotation list
rotation_list = Rotation.from_euler(np.arange(0, 360, 1), degrees = True)

In [41]:
# Create a structure library
struc_lib = StructureLibrary(['phase_aFe', 'phase_FeC', 'phase_MnS'], [phase_aFe, phase_FeC, phase_MnS], [rotation_list]*3)


NameError: name 'phase_aFe' is not defined

In [ ]:
uvw_aFe = [1, 1, 1]
uvw_FeC = [0, 0, 1]
uvw_MnS = [1, 1, 1]

Define how far from the zone axis you want to sample orientations.



In [ ]:
threshold_angle =180

In [ ]:
def reduce_beam_directions(phase, grid, orientations_z, zone_axis, threshold):
    za = Miller(uvw=zone_axis, phase=phase)
    angles = orientations_z.angle_with(za)
    reduced_grid = grid[angles <= np.deg2rad(threshold)]

    return reduced_grid

In [ ]:
grid_aFe_zone = reduce_beam_directions(
    phase_aFe, grid_aFe, orientations_aFe_z, uvw_aFe, threshold_angle)

grid_FeC_zone = reduce_beam_directions(
    phase_FeC, grid_FeC, orientations_FeC_z, uvw_FeC, threshold_angle)

grid_MnS_zone = reduce_beam_directions(
    phase_MnS, grid_MnS, orientations_MnS_z, uvw_MnS, threshold_angle)

We have now only sampled the orientations aorund the known zone axis for each phase, which reduced the list of orientations significantly. Compare the length of each grid to the length of the grids sampling the whole fundamental zone.

In [ ]:
print(len(grid_aFe_zone), len(grid_FeC_zone), len(grid_MnS_zone))


## 2d. Simulating kinematical diffraction
Set up lists of the phases, their structures and the grid of orientations.

In [ ]:
rotation_list


In [40]:
# List of phase names:
phases = ['aFe', 'FeC', 'MnS']

# List of structures:
structures = [structure_aFe, structure_FeC, structure_MnS]

# List of rotations:
grids = [grid_aFe_zone, grid_FeC_zone, grid_MnS_zone]

NameError: name 'grid_aFe_zone' is not defined

In [ ]:
library = StructureLibrary(phases, structures, grids)

Define a new structure library containing the phases, structures and grids, which we will use to simulate our diffraction patterns.

## Here the max_excitation_error is used -  beware, it significantly changes the number of diffraction spots one sees

In [ ]:
diff_lib = lib_gen.get_diffraction_library(library,
                                           calibration=calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=half_shape,
                                           with_direct_beam=False,
                                           max_excitation_error= 0.03 )# max_excitation_error) #

## A streamlined version of the simulation code

In [76]:
# Initialize diffraction generator
diff_gen = DiffractionGenerator(accelerating_voltage=200.)

# Initialize library generator
lib_gen = DiffractionLibraryGenerator(diff_gen)

# Define parameters
half_shape = (64, 64)
reciprocal_radius = np.sqrt(64**2*2)*calibration
resolution = 1

# Generate orientation grids
grid_aFe = get_beam_directions_grid(resolution=resolution, crystal_system="cubic")
grid_FeC = get_beam_directions_grid(resolution=resolution, crystal_system="orthorhombic")
grid_MnS = get_beam_directions_grid(resolution=resolution, crystal_system="cubic")

# Define crystal symmetries
symmetry_aFe = symmetry.Oh
symmetry_FeC = symmetry.D2h
symmetry_MnS = symmetry.Oh

# Generate orientations
orientations_aFe = Orientation.from_euler(np.deg2rad(grid_aFe), symmetry=symmetry_aFe)
orientations_FeC = Orientation.from_euler(np.deg2rad(grid_FeC), symmetry=symmetry_FeC)
orientations_MnS = Orientation.from_euler(np.deg2rad(grid_MnS), symmetry=symmetry_MnS)

# Define zone axes
uvw_aFe = [1, 1, 1]
uvw_FeC = [0, 0, 1]
uvw_MnS = [0, 0, 1]

# Define threshold angle
threshold_angle = 25

# Define function to reduce beam directions
def reduce_beam_directions(phase, grid, orientations_z, zone_axis, threshold):
    za = Miller(uvw=zone_axis, phase=phase)
    angles = orientations_z.angle_with(za)
    reduced_grid = grid[angles <= np.deg2rad(threshold)]
    return reduced_grid
    
# Phases    
phase_aFe = Phase(structure=structure_aFe, point_group=symmetry_aFe)
phase_FeC = Phase(structure=structure_FeC, point_group=symmetry_FeC)
phase_MnS = Phase(structure=structure_MnS, point_group=symmetry_MnS)

orientations_aFe_z = orientations_aFe * Vector3d.zvector() 
orientations_FeC_z = orientations_FeC * Vector3d.zvector()
orientations_MnS_z = orientations_MnS * Vector3d.zvector()

# Reduce orientation grids
grid_aFe_zone = reduce_beam_directions(phase_aFe, grid_aFe, orientations_aFe_z, uvw_aFe, threshold_angle)
grid_FeC_zone = reduce_beam_directions(phase_FeC, grid_FeC, orientations_FeC_z, uvw_FeC, threshold_angle)
grid_MnS_zone = reduce_beam_directions(phase_MnS, grid_MnS, orientations_MnS_z, uvw_MnS, threshold_angle)

# Create structure library
library = StructureLibrary(['aFe', 'FeC', 'MnS'], [structure_aFe, structure_FeC, structure_MnS], [grid_aFe_zone, grid_FeC_zone, grid_MnS_zone])

# Generate diffraction library
diff_lib = lib_gen.get_diffraction_library(library, calibration=calibration, reciprocal_radius=reciprocal_radius, half_shape=half_shape, with_direct_beam=False, max_excitation_error=0.045)


C:\Users\ursulal\AppData\Local\miniconda3\envs\2312_hspy\Lib\site-packages\diffsims\generators\sphere_mesh_generators.py:514: RuntimeWarning: invalid value encountered in divide
  phi2 = sign * np.nan_to_num(np.arccos(x_comp / norm_proj))


In [74]:
# Access the orientations for each simulation
for phase, entries in diff_lib.items():
    for euler_angles_str, simulations in entries.items():
        if isinstance(euler_angles_str, str):
            print(f"Phase: {phase}, Euler angles: {euler_angles_str}")
         


Phase: aFe, Euler angles: simulations
Phase: aFe, Euler angles: orientations
Phase: aFe, Euler angles: pixel_coords
Phase: aFe, Euler angles: intensities
Phase: FeC, Euler angles: simulations
Phase: FeC, Euler angles: orientations
Phase: FeC, Euler angles: pixel_coords
Phase: FeC, Euler angles: intensities
Phase: MnS, Euler angles: simulations
Phase: MnS, Euler angles: orientations
Phase: MnS, Euler angles: pixel_coords
Phase: MnS, Euler angles: intensities


## Let's play a bit with the values of the max_exitation-error

In [ ]:
diff_lib['aFe']['simulations'].c

The diffraction library is a dictionary, which takes the phase names as key. The library of each phase can then be accessed using diff_lib['key']. Here we print the keys we have available.

In [ ]:
diff_lib.keys() 

Plot the simulation closest to zone-axis.



In [ ]:
diff_lib['aFe']['simulations'][0].plot() # Closest to Euler angle (0, 0, 90)
#diff_lib['FeC']['simulations'][0].plot() # Closest to Euler angle (0, 90, 90)
#diff_lib['MnS']['simulations'][0].plot() # Closest to Euler angle (0, 54.4, 0)

In [ ]:
fig, axs = plt.subplots()
axs.set_aspect(1)
diff_lib['aFe']['simulations'][0].plot(ax=axs)

## Here an attempt to make more realistic looking simulated diff patterns 
240312 on 240320 it works! :)

In [ ]:
diff_lib['aFe']['simulations'][0].

## Here the code for creating Signal2D  objects from simulated data, that look more like real SPED data 
Here I use object oriented style so it is easier to work with simulations of different structures in the same dataset. The functions in the class are independent of the simulated data.

In [46]:
class GaussianPattern:
    def __init__(self, peak_diam, max_avg_int, extra_padding):
        self.peak_diam = peak_diam
        self.max_avg_int = max_avg_int
        self.extra_padding = extra_padding
        self.patterns = []

    def create_gaussian(self, size, _):
        sigma = size / 3.0
        x = np.linspace(-size / 2, size / 2, size)
        y = np.linspace(-size / 2, size / 2, size)
        x, y = np.meshgrid(x, y)
        gaussian = self.max_avg_int * np.exp(-(x**2 + y**2) / (2.0 * sigma**2))
        window = np.outer(np.hanning(size), np.hanning(size))
        gaussian *= window
        return gaussian

    def add_pattern(self, simulation):
        max_coord = np.max(simulation.calibrated_coordinates)
        min_coord = np.min(simulation.calibrated_coordinates)
        pattern_size = int(max_coord - min_coord + 2*self.peak_diam + self.extra_padding)
        modified_pattern = np.zeros((pattern_size, pattern_size))
        coordinates = simulation.calibrated_coordinates - min_coord + 1.5*self.extra_padding
        for peak in coordinates:
            center_x = int(peak[0])
            center_y = int(peak[1])
            start_x = max(center_x - self.peak_diam // 2, 0)
            stop_x = min(center_x + self.peak_diam // 2, pattern_size)
            start_y = max(center_y - self.peak_diam // 2, 0)
            stop_y = min(center_y + self.peak_diam // 2, pattern_size)
            region_size = (stop_x - start_x, stop_y - start_y)
            gaussian = self.create_gaussian(region_size[1], region_size[0])
            modified_pattern[start_x:stop_x, start_y:stop_y] += gaussian
        self.patterns.append(modified_pattern)

    def pad_patterns(self):
        max_pattern_size = max(max(pattern.shape) for pattern in self.patterns)
        return [np.pad(pattern, ((0, max_pattern_size - pattern.shape[0]), 
                                  (0, max_pattern_size - pattern.shape[1]))) 
                for pattern in self.patterns]

    def to_signal(self):
        padded_patterns_array = np.array(self.pad_patterns())
        return Signal2D(padded_patterns_array)



Here we execute and plot the simulated data of different structures:

In [66]:
# Usage:
gaussian_pattern = GaussianPattern(peak_diam=20, max_avg_int=20, extra_padding=20)
for sim in diff_lib['aFe']['simulations']:
    gaussian_pattern.add_pattern(sim)
signal = gaussian_pattern.to_signal()
signal.plot()


In [67]:
# Usage:
gaussian_pattern = GaussianPattern(peak_diam=20, max_avg_int=20, extra_padding=20)
for sim in diff_lib['FeC']['simulations']:
    gaussian_pattern.add_pattern(sim)
signal = gaussian_pattern.to_signal()
signal.plot()


In [77]:
# Usage:
gaussian_pattern = GaussianPattern(peak_diam=20, max_avg_int=20, extra_padding=20)
for sim in diff_lib['MnS']['simulations']:
    gaussian_pattern.add_pattern(sim)
signal = gaussian_pattern.to_signal()
signal.plot()


In [ ]:
modified_patterns_signal

In [ ]:
sim.calibrated_coordinates

In [ ]:
sim.calibrated_coordinates

In [ ]:
%matplotlib qt
# Convert the list of modified patterns to a 3D numpy array
modified_patterns_array = np.dstack(modified_patterns)

# Transpose the array to get the correct orientation
modified_patterns_array = np.transpose(modified_patterns_array, (2, 0, 1))

# Convert the 3D array to a Signal2D object
signal = Signal2D(modified_patterns_array)

# Plot the signal
signal.plot()


In [ ]:
modified_patterns

In [ ]:
simulation.calibrated_coordinates

# 3. Template matching

## 3a. Pre-processing
The SPED dataset has already been centered and calibrated, which is crucial as pre-processing steps before template matching. Depending on the state of your TEM and detection system, distortion correction can also be crucial before the template matching, but this was not done here.

Due to the weak reflections, a log-shift transform can be applied to all experimental patterns prior to template matching. We define this as a new function. The small added shift, a, amplifies weak reflection but also noise and must be used with caution depending on the dataset.
set.

In [ ]:
def log_shift(dp):
    a = 0.001*np.max(dp.max().data)
    log_shift = np.log10(dp+a) - np.log10(a)
    return log_shift

Using the map function, the log_shift function can easily be applied to each pattern.



In [ ]:
dp_log_shift = dp.map(function=log_shift, inplace=False)


We can plot the signals side by side using the same navigator to easily see the difference between the original and the signal after applying log shift. Note also the difference in the navigation images, with less strain contrast and more "high angle"-like contrast appearing after applying the log_shift.

In [ ]:
hs.plot.plot_signals([dp, dp_log_shift], cmap='magma_r')


## 3b. Normalised cross correlation
We define the maximum radius that will be considered in the cross correlation for template matching. By default, this will be the length from the center of a pattern to the corner (half diagonal). We set it to half the size of the pattern, to avoid using the highest scattering angles where the noise is largest. This variable has units of pixels.

In [ ]:
max_r = 50 # Is binning an option? from 256 -> 128


Perform the template matching, including finding the in-plane rotations.



In [ ]:
result, phasedict = iutls.index_dataset_with_template_rotation(
    dp_log_shift, diff_lib, max_r=max_r) 

# 4. Visualising the results from template matching
We define a crystal map that contains the results from the template matching.




In [ ]:
xmap = iutls.results_dict_to_crystal_map(
    results=result, phase_key_dict=phasedict, diffraction_library=diff_lib)


We can plot the crystal map, which will plot the phase map.



In [ ]:
xmap.plot()

In [ ]:
xmap


Before we can plot the orientations, we must define the symmetries of each phases contained within the crystal map.



In [ ]:
#xmap.phases["aFe"].point_group = symmetry_aFe
xmap.phases["FeC"].point_group = symmetry_FeC
#xmap.phases["MnS"].point_group = symmetry_MnS

Define the IPF colours that will be used to plot the orientations of each phase.



In [ ]:
from orix import plot


In [ ]:
#key_aFe = plot.IPFColorKeyTSL(symmetry_aFe)
key_FeC = plot.IPFColorKeyTSL(symmetry_FeC)
#key_MnS = plot.IPFColorKeyTSL(symmetry_MnS)

Plot the IPF colour keys



In [ ]:
key_aFe.plot()
#key_FeC.plot()
#key_MnS.plot()

In [ ]:
colours_aFe = key_aFe.orientation2color(xmap["aFe"].orientations)
#colours_FeC = key_FeC.orientation2color(xmap["FeC"].orientations)
#colours_MnS = key_MnS.orientation2color(xmap["MnS"].orientations)


Plot the orientations per phase with the given colour keys from the IPFs.

 

In [ ]:
xmap['aFe'].plot(colours_aFe)
#xmap['FeC'].plot(colours_FeC)
#xmap['MnS'].plot(colours_MnS)


Plot the correlation scores of the best matches. Identify areas where the template matching did not get a high correlation score.



In [ ]:
xmap.plot('correlation', colorbar=True)


In the future, there should be a function allowing to plot the best matching template on top of the PED pattern in each pixel of the SPED scan. That would be an extremenely useful tool when we want to check the results visually.

For now, we will do this for one pattern at a time.

In [ ]:
i = 0
j = 5#60
dp_i = dp.inav[i, j]
template_index = xmap.get_map_data(xmap.template_index)[i, j]
phase_index = xmap.get_map_data('phase_id')[i, j]
simulation_i = diff_lib[phasedict[phase_index]]['simulations'][template_index]

In [ ]:
%matplotlib inline
a_i, c_i = iutls.get_in_plane_rotation_correlation(dp_i.data, simulation_i)
in_plane_angle = a_i[np.argmax(c_i)]
in_plane_angle

In [ ]:
putls.plot_template_over_pattern(dp_i.data,
                                 simulation_i,
                                 in_plane_angle=in_plane_angle,
                                 norm=LogNorm())